In [1]:
import cv2
import glob
import numpy as np
from skimage.feature import hog
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier

# Definir las carpetas de frutas frescas y podridas
freshfruits = {
    "freshapples": "freshapple",
    "freshbanana": "freshbanana",
    "freshoranges": "freshorange"
}

rottenfruits = {
    "rottenapples": "rottenapple",
    "rottenbanana": "rottenbanana",
    "rottenoranges": "rottenorange"
}

# Inicializar listas para almacenar características y etiquetas
data_train = []
labels_train = []
data_test = []
labels_test = []

# Función para procesar imágenes y extraer características HOG y histogramas de color
def process_images(folder_dict, base_path, data_list, labels_list):
    for folder, label in folder_dict.items():
        folder_path = f"{base_path}/{folder}/*.png"
        for image_path in glob.glob(folder_path):
            img = cv2.imread(image_path)
            if img is None or len(img.shape) != 3 or img.shape[2] != 3:
                print(f"Imagen no válida o con canales incorrectos: {image_path}")
                continue
            try:
                # Redimensionar la imagen
                resized_img = cv2.resize(img, (64, 32))
                # Conversión a escala de grises para HOG
                gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                # Extracción de características HOG
                hog_features = hog(
                    gray_img,
                    orientations=9,
                    pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2),
                    visualize=False,
                    feature_vector=True,
                    block_norm='L2-Hys'
                )
                # Extracción de histogramas de color
                hist_features = []
                for i in range(3):  # Canales B, G, R
                    hist = cv2.calcHist([resized_img], [i], None, [256], [0, 256])
                    hist = cv2.normalize(hist, hist).flatten()
                    hist_features.extend(hist)
                # Concatenar características HOG y histogramas de color
                features = np.concatenate([hog_features, hist_features])
            except Exception as e:
                print(f"Error procesando {image_path}: {e}")
                continue

            # Agregar las características y la etiqueta a las listas
            data_list.append(features)
            labels_list.append(label)

# Procesar imágenes de entrenamiento
train_base_path = ".gitignore/dataset/train"
process_images(freshfruits, train_base_path, data_train, labels_train)
process_images(rottenfruits, train_base_path, data_train, labels_train)

print("Total de imágenes de entrenamiento procesadas:", len(data_train))

# Procesar imágenes de prueba
test_base_path = ".gitignore/dataset/test"
process_images(freshfruits, test_base_path, data_test, labels_test)
process_images(rottenfruits, test_base_path, data_test, labels_test)

print("Total de imágenes de test procesadas:", len(data_test))

# Convertir las características y etiquetas a arrays de NumPy
X_train = np.array(data_train)
y_train_labels = np.array(labels_train)
X_test = np.array(data_test)
y_test_labels = np.array(labels_test)

# Codificar las etiquetas de forma consistente
encoder = LabelEncoder()
encoder.fit(np.concatenate((y_train_labels, y_test_labels)))

y_train = encoder.transform(y_train_labels)
y_test = encoder.transform(y_test_labels)

Total de imágenes de entrenamiento procesadas: 10901
Total de imágenes de test procesadas: 2698


In [2]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model One-vs-One amb kernel lineal
ovo_classifier = OneVsOneClassifier(SVC(kernel='linear', probability=True, random_state=42))
ovo_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred = ovo_classifier.predict(X_test)

# Avaluar el model
accuracy = accuracy_score(y_test, y_pred)
print("----- One-vs-One (Kernel Lineal) -----")
print(f"Accuracy: {accuracy:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred))

----- One-vs-One (Kernel Lineal) -----
Accuracy: 0.8944

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.87      0.85      0.86       395
 freshbanana       0.95      0.94      0.95       381
 freshorange       0.95      0.91      0.93       388
 rottenapple       0.82      0.87      0.85       601
rottenbanana       0.95      0.95      0.95       530
rottenorange       0.85      0.82      0.84       403

    accuracy                           0.89      2698
   macro avg       0.90      0.89      0.90      2698
weighted avg       0.90      0.89      0.89      2698

Matriu de Confusió:
[[336   1   1  50   1   6]
 [  0 360   0   2  18   1]
 [  7   1 354   6   1  19]
 [ 41   0   4 525   2  29]
 [  0  16   0   5 506   3]
 [  2   0  14  49   6 332]]


In [3]:
# One-vs-Rest (Kernel Lineal) - Model de Classificació

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model One-vs-Rest amb kernel lineal
ovr_classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True, random_state=42))
ovr_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred_ovr = ovr_classifier.predict(X_test)

# Avaluar el model
accuracy_ovr = accuracy_score(y_test, y_pred_ovr)
print("----- One-vs-Rest (Kernel Lineal) -----")
print(f"Accuracy: {accuracy_ovr:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred_ovr, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred_ovr))

----- One-vs-Rest (Kernel Lineal) -----
Accuracy: 0.8747

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.86      0.84      0.85       395
 freshbanana       0.92      0.92      0.92       381
 freshorange       0.90      0.92      0.91       388
 rottenapple       0.80      0.87      0.83       601
rottenbanana       0.92      0.93      0.93       530
rottenorange       0.87      0.76      0.81       403

    accuracy                           0.87      2698
   macro avg       0.88      0.87      0.88      2698
weighted avg       0.88      0.87      0.87      2698

Matriu de Confusió:
[[333   4   6  48   2   2]
 [  4 351   0   2  22   2]
 [  4   2 357  12   3  10]
 [ 40   4   9 520   4  24]
 [  2  20   0   6 494   8]
 [  2   0  23  61  12 305]]


In [4]:
# SVM amb Kernel RBF - Model de Classificació

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model SVM amb kernel RBF
rbf_classifier = SVC(kernel='rbf', gamma='scale', probability=True, random_state=42)
rbf_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred_rbf = rbf_classifier.predict(X_test)

# Avaluar el model
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
print("----- SVM amb Kernel RBF -----")
print(f"Accuracy: {accuracy_rbf:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred_rbf, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred_rbf))

----- SVM amb Kernel RBF -----
Accuracy: 0.8769

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.87      0.84      0.85       395
 freshbanana       0.96      0.91      0.93       381
 freshorange       0.93      0.89      0.91       388
 rottenapple       0.79      0.90      0.84       601
rottenbanana       0.90      0.95      0.92       530
rottenorange       0.87      0.75      0.80       403

    accuracy                           0.88      2698
   macro avg       0.89      0.87      0.88      2698
weighted avg       0.88      0.88      0.88      2698

Matriu de Confusió:
[[330   1   1  58   0   5]
 [  0 347   0   1  32   1]
 [  8   1 344   8  14  13]
 [ 34   0   3 539   2  23]
 [  1  14   0   8 505   2]
 [  5   0  20  67  10 301]]


In [5]:
# SVM amb Kernel Polinòmic - Model de Classificació

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model SVM amb kernel polinòmic
poly_classifier = SVC(kernel='poly', degree=3, coef0=1, probability=True, random_state=42)
poly_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred_poly = poly_classifier.predict(X_test)

# Avaluar el model
accuracy_poly = accuracy_score(y_test, y_pred_poly)
print("----- SVM amb Kernel Polinòmic -----")
print(f"Accuracy: {accuracy_poly:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred_poly, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred_poly))

----- SVM amb Kernel Polinòmic -----
Accuracy: 0.9411

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.92      0.91      0.91       395
 freshbanana       0.98      0.97      0.97       381
 freshorange       0.98      0.97      0.97       388
 rottenapple       0.89      0.93      0.91       601
rottenbanana       0.98      0.98      0.98       530
rottenorange       0.92      0.89      0.90       403

    accuracy                           0.94      2698
   macro avg       0.94      0.94      0.94      2698
weighted avg       0.94      0.94      0.94      2698

Matriu de Confusió:
[[359   1   0  32   0   3]
 [  0 369   0   1   9   2]
 [  3   0 375   3   0   7]
 [ 23   0   2 556   1  19]
 [  1   7   0   0 521   1]
 [  4   0   6  32   2 359]]
